In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')

from IPython.core.display import Image
import uuid 

import gc

In [2]:
#R call

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [3]:
#Call custom Python module
import parametric as param

In [4]:
def is_Hermitian(m):
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False

In [20]:
def fastclime_py(x,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
    
    for i in range(maxnlambda):
        #Extract every other element in each column of iicov before reshaping
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    

In [6]:
#Generate data

L = fastclime.fastclime_generator(n = 40, d = 10)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [33]:
Pyout = fastclime_py(pydat,0.1)

In [8]:
#Compare results to R

In [34]:
#%timeit -n1 -r1 fastclime.fastclime(L.rx2('data'),0.1)
Rout = fastclime.fastclime(L.rx2('data'),0.1)

Allocating memory 
start recovering 
preparing precision and path matrix list 
Done! 


In [10]:
###Draw network graph

# """
# Draw a graph with matplotlib.
# You must have matplotlib for this to work.
# """
# try:
#     import matplotlib.pyplot as plt
# except:
#     raise

# import networkx as nx

# G=nx.path_graph(8)
# nx.draw(G)
# plt.savefig("simple_path.png") # save as png
# plt.show() # display